## Experiments


In [47]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser 
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

In [48]:
load_dotenv()

True

In [49]:
# Prompt Engineering

system_template = (
    "You are an interviewer for a technical position, conducting a Data Structures and Algorithms (DSA) interview."
    "The candidate has been given the following problem to solve: \n{challenge}\n"
    "Your responsibilities include presenting the problem, tracking the time allotted ({total_time} minutes), assessing the candidate's approach, and evaluating their solution."
    "The candidate is actively working on the problem and may request hints or clarifications. They will notify you when they are ready to submit their solution."
    "Please remind the candidate to manage their time effectively within the allotted time."
    "REMINDER: Use Interview Conversational Tone and provide constructive feedback to the candidate."
)

candidate_template = (
    "Current Time: {current_time} minutes\n"
    "Current User's Attempted Code: \n```{current_attempt}```\n"
    "If the user is ready to submit their solution, they should type 'submit'. Otherwise, they can request hints or clarifications as long as they have time remaining."
    "If the user runs out of time, the interviewer will assess their current solution and provide feedback."
    "If the user submits their solution before time runs out, the interviewer will evaluate their solution and provide feedback."
    "Consider the following dimensions when evaluating their solution: Problem-Solving Approach Algorithmic Efficiency, Code Quality, Efficiency in Implementation, Communication, Time Management, Adaptability, Overall Performance"
)

# Variables of the template:
# {challenge} - the problem statement
# {total_time} - the total time allotted for the interview
# {current_time} - the current time elapsed in the interview
# {current_attempt} - the current code attempt by the candidate
# {chat_history} - the chat history between the interviewer and the candidate

interviewer_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        MessagesPlaceholder("chat_history"),
        ("human", candidate_template),
    ]
)

In [50]:
llm = ChatOpenAI(model="gpt-4o-mini")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [58]:
interview_chain = interviewer_template | llm | StrOutputParser()

In [59]:
problem = """
27. Remove Element (Easy)

Given an integer array nums and an integer val, remove all occurrences of val in nums in-place. The order of the elements may be changed. Then return the number of elements in nums which are not equal to val.

Consider the number of elements in nums which are not equal to val be k, to get accepted, you need to do the following things:

Change the array nums such that the first k elements of nums contain the elements which are not equal to val. The remaining elements of nums are not important as well as the size of nums.
Return k.

You can use any programming language to solve this problem.
"""

total_time = 45

print("Welcome to the DSA Interview Simulator!")


Welcome to the DSA Interview Simulator!


In [61]:
round = 0
print(f"Hello, You have to solve the following problem in {total_time} minutes: \n{problem}\n\n")
while True:
    current_time = int(input(f"[{round}] Current Time (You enter for now): "))
    current_attempt = input(f"[{round}] Your Attempt: \
                            \n")
    if current_attempt == "submit":
        current_attempt = "The candidate has submitted their solution. Please check the history and evaluate their final submission."
        variables = {"challenge": problem, "total_time": total_time, "current_time": total_time, "current_attempt": current_attempt, "chat_history": memory.buffer_as_messages}
    else:
        variables = {"challenge": problem, "total_time": total_time, "current_time": current_time, "current_attempt": current_attempt, "chat_history": memory.buffer_as_messages}
     
    ai = interview_chain.invoke(variables)
    print("AI: ", ai)
    input("Press Enter to continue...")
    memory.chat_memory.add_user_message(f"Time: {current_time}/{total_time} minutes \n\n Attempt: \n ```{current_attempt}```") 
    memory.chat_memory.add_ai_message(ai)
    


    round += 1

Hello, You have to solve the following problem in 45 minutes: 

27. Remove Element (Easy)

Given an integer array nums and an integer val, remove all occurrences of val in nums in-place. The order of the elements may be changed. Then return the number of elements in nums which are not equal to val.

Consider the number of elements in nums which are not equal to val be k, to get accepted, you need to do the following things:

Change the array nums such that the first k elements of nums contain the elements which are not equal to val. The remaining elements of nums are not important as well as the size of nums.
Return k.

You can use any programming language to solve this problem.



AI:  It looks like you've just printed "Hello World" as your attempted code. That's a good start if you were looking to check your environment, but we need to focus on solving the problem at hand about removing elements from the array.

If you need any hints or clarifications about the problem, feel free to 

ValueError: invalid literal for int() with base 10: ''